In [37]:
import pandas as pd
import numpy as np

# a) Generate a Dataset

We require a dataset of randomly generated labels. There should be 8 features and 50,000 examples.

Randomly generate an array of integers from 0-2

In [38]:
rand_arr = np.round(np.random.rand(50000,8)*3).astype(int)
rand_arr

array([[2, 2, 1, ..., 3, 1, 1],
       [0, 2, 0, ..., 1, 3, 1],
       [3, 2, 1, ..., 2, 2, 1],
       ...,
       [2, 1, 1, ..., 3, 2, 0],
       [1, 1, 2, ..., 2, 1, 1],
       [3, 3, 3, ..., 2, 2, 2]])

Convert this to a Pandas dataframe.

In [39]:
df = pd.DataFrame(rand_arr)
df

,0,1,2,3,4,5,6,7
0,2,2,1,2,1,3,1,1
1,0,2,0,1,1,1,3,1
2,3,2,1,2,0,2,2,1
3,3,2,2,2,2,2,0,2
4,0,2,1,1,3,1,0,1
...,...,...,...,...,...,...,...,...
49995,1,1,1,2,1,1,2,1
49996,2,1,1,1,1,2,3,1
49997,2,1,1,1,0,3,2,0
49998,1,1,2,2,2,2,1,1
